In [ ]:
import hiddenlayer as hl
import torch
from transformers import MistralForSequenceClassification, LlamaTokenizerFast

In [2]:
hf_repo = 'h2oai/h2o-danube2-1.8b-base'
device = 'cpu'

In [3]:
model = MistralForSequenceClassification.from_pretrained(hf_repo, num_labels=3, device_map=device,
                                                         torch_dtype=torch.bfloat16)

tokenizer = LlamaTokenizerFast.from_pretrained(hf_repo)

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at h2oai/h2o-danube2-1.8b-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:

if tokenizer.pad_token is None:
    num_added_toks = tokenizer.add_special_tokens({'pad_token': '<|pad_token|>'})
    assert num_added_toks == 1
    model.resize_token_embeddings(len(tokenizer))
    model.config.pad_token_id = tokenizer.pad_token_id


In [6]:

premises = [
    "and",
]

hypotheses = [
    'my'
]

inputs = tokenizer(premises, hypotheses, truncation='only_first', padding="max_length",
                    max_length=25, add_special_tokens=True, return_tensors='pt')

inputs

{'input_ids': tensor([[32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
         32000, 32000, 32000,   304,   586]]), 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         1]])}

In [7]:
from torchview import draw_graph

model = MLP()
batch_size = 2
# device='meta' -> no memory is consumed for visualization
model_graph = draw_graph(model, input_size=(batch_size, 128), device='meta')
model_graph.visual_graph

C:\Users\lachl\PycharmProjects\bart-large-mnli-recreation\.venv\lib\site-packages\transformers\modeling_attn_mask_utils.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
C:\Users\lachl\PycharmProjects\bart-large-mnli-recreation\.venv\lib\site-packages\transformers\modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
C:\Users\lachl\PycharmProjects\bart-large-mnli-recreation\.venv\lib\site-packages\transformers\models\mistral\modeling_mi

AttributeError: module 'torch.onnx' has no attribute '_optimize_trace'

In [ ]:
# make_dot(model_outputs[0].mean(), params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")